In [ ]:
!pip install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.1 MB/s eta 0:00:00


In [ ]:
!pip install py-readability-metrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import nltk
from curses.ascii import isdigit
from nltk.corpus import cmudict
from readability import Readability
import textstat
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, log_loss
from scipy.special import softmax
from nlp import load_dataset
from nlp import Dataset
import string
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
nltk.download('cmudict')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
in_df = pd.read_csv('init_re.csv')
train_df = pd.read_csv('data_bio_read.csv')

my_df = pd.concat([train_df,in_df])
my_df = my_df.reset_index(drop = True)
my_df = my_df.dropna()
my_df

,index_article,index_paragraph,content,label
0,1.0,1.0,Neurogenesis is a strictly controlled process ...,1.0
1,1.0,2.0,The potential of involving adult neurogenesis ...,1.0
2,1.0,3.0,To remedy this absence of suitable tool for ne...,1.0
3,1.0,4.0,A 2380-bp Sal I-Not I fragment of pCAG-CreERT2...,1.0
4,1.0,5.0,"The targeting plasmid, phuDCX-3509-CreERT2-3’-...",1.0
...,...,...,...,...
784,13.0,23.0,Microtiter plates from Nunc in Denmark were us...,0.0
785,13.0,24.0,Splenocytes from mice were collected after the...,0.0
786,13.0,25.0,"On day 3 after the final vaccination, the cerv...",0.0
787,13.0,26.0,After taking out the lungs of mice six days af...,0.0


In [ ]:
# my_df = pd.read_csv('train_hand.csv', encoding = 'gb18030')

my_df['unique_words_count'] = 0
my_df['avg_unique_words_per_sentence'] = 0
my_df['upper_case_pct'] = 0
my_df['pos_tag_per_para'] = 0
my_df['avg_pos_tag_per_sen'] = 0
my_df['pos_to_unique_ratio'] = 0
my_df['syllabus_pct'] = 0
my_df['fk_grade'] = 0


In [ ]:
val_df = pd.read_csv('val_bio_data.csv')
test_df = pd.read_csv('test_bio_data.csv')

In [ ]:
def fk_grade(words, sentences, syl):
  return 0.39 * (words/sentences) + 11.8 * syl - 15.59

In [ ]:
for i in range(len(my_df)):
  text = my_df['content'].iloc[i]
  words = text.split()
  sens = nltk.sent_tokenize(text)#分句指令
  num_upper = 0
  syl = 0

  for j in words:
    syl += textstat.syllable_count(j)
    if j.lower() != j:
      num_upper += 1

  my_df['unique_words_count'].iloc[i] = len(set(words))# unique_words
  my_df['avg_unique_words_per_sentence'].iloc[i] = my_df['unique_words_count'].iloc[i]/len(sens)
  my_df['upper_case_pct'].iloc[i] = num_upper
  my_df['pos_tag_per_para'].iloc[i] = len(set(dict(nltk.pos_tag(words)).values()))
  my_df['avg_pos_tag_per_sen'].iloc[i] = my_df['pos_tag_per_para'].iloc[i]/len(sens)
  my_df['pos_to_unique_ratio'].iloc[i] = my_df['pos_tag_per_para'].iloc[i]/my_df['unique_words_count'].iloc[i]
  my_df['syllabus_pct'].iloc[i] = syl/len(words)
  my_df['fk_grade'].iloc[i] = fk_grade(len(words),len(sens),my_df['syllabus_pct'].iloc[i])


my_df

<ipython-input-103-60d20ac32428>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df['unique_words_count'].iloc[i] = len(set(words))# unique_words
<ipython-input-103-60d20ac32428>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df['avg_unique_words_per_sentence'].iloc[i] = my_df['unique_words_count'].iloc[i]/len(sens)
<ipython-input-103-60d20ac32428>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df['upper_case_pct'].ilo

,index_article,index_paragraph,content,label,unique_words_count,avg_unique_words_per_sentence,upper_case_pct,pos_tag_per_para,avg_pos_tag_per_sen,pos_to_unique_ratio,syllabus_pct,fk_grade
0,1.0,1.0,Neurogenesis is a strictly controlled process ...,1.0,115,16.428571,15,18,2.571429,0.156522,1.994012,17.243627
1,1.0,2.0,The potential of involving adult neurogenesis ...,1.0,120,20.000000,14,17,2.833333,0.141667,1.873563,17.828046
2,1.0,3.0,To remedy this absence of suitable tool for ne...,1.0,62,20.666667,5,16,5.333333,0.258065,1.756098,15.791951
3,1.0,4.0,A 2380-bp Sal I-Not I fragment of pCAG-CreERT2...,1.0,87,10.875000,51,18,2.250000,0.206897,1.643357,10.772858
4,1.0,5.0,"The targeting plasmid, phuDCX-3509-CreERT2-3’-...",1.0,117,11.700000,49,14,1.400000,0.119658,1.589595,9.914225
...,...,...,...,...,...,...,...,...,...,...,...,...
784,13.0,23.0,Microtiter plates from Nunc in Denmark were us...,0.0,100,20.000000,31,17,3.400000,0.170000,1.507576,12.495394
785,13.0,24.0,Splenocytes from mice were collected after the...,0.0,41,13.666667,9,15,5.000000,0.365854,1.600000,9.790000
786,13.0,25.0,"On day 3 after the final vaccination, the cerv...",0.0,57,14.250000,15,13,3.250000,0.228070,1.841270,12.279484
787,13.0,26.0,After taking out the lungs of mice six days af...,0.0,93,15.500000,18,17,2.833333,0.182796,1.669565,11.585870


In [ ]:
val_df['unique_words_count'] = 0
val_df['avg_unique_words_per_sentence'] = 0
val_df['upper_case_pct'] = 0
val_df['pos_tag_per_para'] = 0
val_df['avg_pos_tag_per_sen'] = 0
val_df['pos_to_unique_ratio'] = 0
val_df['syllabus_pct'] = 0
val_df['fk_grade'] = 0


for i in range(len(val_df)):
  text = val_df['content'].iloc[i]
  words = text.split()
  sens = nltk.sent_tokenize(text)#分句指令
  num_upper = 0
  syl = 0

  for j in words:
    syl += textstat.syllable_count(j)
    if j.lower() != j:
      num_upper += 1

  val_df['unique_words_count'].iloc[i] = len(set(words))# unique_words
  val_df['avg_unique_words_per_sentence'].iloc[i] = val_df['unique_words_count'].iloc[i]/len(sens)
  val_df['upper_case_pct'].iloc[i] = num_upper
  val_df['pos_tag_per_para'].iloc[i] = len(set(dict(nltk.pos_tag(words)).values()))
  val_df['avg_pos_tag_per_sen'].iloc[i] = val_df['pos_tag_per_para'].iloc[i]/len(sens)
  val_df['pos_to_unique_ratio'].iloc[i] = val_df['pos_tag_per_para'].iloc[i]/val_df['unique_words_count'].iloc[i]
  val_df['syllabus_pct'].iloc[i] = syl/len(words)
  val_df['fk_grade'].iloc[i] = fk_grade(len(words),len(sens),val_df['syllabus_pct'].iloc[i])


val_df

<ipython-input-104-b6bd441049d7>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df['unique_words_count'].iloc[i] = len(set(words))# unique_words
<ipython-input-104-b6bd441049d7>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df['avg_unique_words_per_sentence'].iloc[i] = val_df['unique_words_count'].iloc[i]/len(sens)
<ipython-input-104-b6bd441049d7>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df['upper_case_pct']

,index_article,index_paragraph,content,label,unique_words_count,avg_unique_words_per_sentence,upper_case_pct,pos_tag_per_para,avg_pos_tag_per_sen,pos_to_unique_ratio,syllabus_pct,fk_grade
0,15,1,"In the last two decades, corticomuscular synch...",1,39,19.500000,5,11,5.500000,0.282051,2.021739,17.236522
1,15,2,Several investigations focused on the mechanis...,1,139,17.375000,24,20,2.500000,0.143885,1.800000,15.156250
2,15,3,We addressed this question investigating the C...,1,36,36.000000,4,15,15.000000,0.416667,1.923077,22.312308
3,15,4,"We tested the following predictions: First, ba...",1,48,48.000000,4,18,18.000000,0.375000,1.981481,28.851481
4,15,5,"Second, recent studies on corticospinal intera...",1,55,18.333333,7,16,5.333333,0.290909,1.666667,13.046667
...,...,...,...,...,...,...,...,...,...,...,...,...
137,18,13,Tissue slides from the tumors were made and su...,0,42,14.000000,10,14,4.666667,0.333333,1.750000,12.860000
138,18,14,The hybridisation reaction was conducted twice...,0,119,23.800000,31,16,3.200000,0.134454,1.626582,15.927671
139,18,15,The study conducted real-time RT-PCR for 10 mi...,0,67,13.400000,25,15,3.000000,0.223881,1.582418,10.180527
140,18,16,The miRNAs were analyzed through various onlin...,0,124,24.800000,52,18,3.600000,0.145161,1.886228,19.693485


In [ ]:
test_df['unique_words_count'] = 0
test_df['avg_unique_words_per_sentence'] = 0
test_df['upper_case_pct'] = 0
test_df['pos_tag_per_para'] = 0
test_df['avg_pos_tag_per_sen'] = 0
test_df['pos_to_unique_ratio'] = 0
test_df['syllabus_pct'] = 0
test_df['fk_grade'] = 0


for i in range(len(test_df)):
  text = test_df['content'].iloc[i]
  words = text.split()
  sens = nltk.sent_tokenize(text)#分句指令
  num_upper = 0
  syl = 0

  for j in words:
    syl += textstat.syllable_count(j)
    if j.lower() != j:
      num_upper += 1

  test_df['unique_words_count'].iloc[i] = len(set(words))# unique_words
  test_df['avg_unique_words_per_sentence'].iloc[i] = test_df['unique_words_count'].iloc[i]/len(sens)
  test_df['upper_case_pct'].iloc[i] = num_upper
  test_df['pos_tag_per_para'].iloc[i] = len(set(dict(nltk.pos_tag(words)).values()))
  test_df['avg_pos_tag_per_sen'].iloc[i] = test_df['pos_tag_per_para'].iloc[i]/len(sens)
  test_df['pos_to_unique_ratio'].iloc[i] = test_df['pos_tag_per_para'].iloc[i]/test_df['unique_words_count'].iloc[i]
  test_df['syllabus_pct'].iloc[i] = syl/len(words)
  test_df['fk_grade'].iloc[i] = fk_grade(len(words),len(sens),test_df['syllabus_pct'].iloc[i])


test_df

<ipython-input-105-e33da521c9b9>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['unique_words_count'].iloc[i] = len(set(words))# unique_words
<ipython-input-105-e33da521c9b9>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['avg_unique_words_per_sentence'].iloc[i] = test_df['unique_words_count'].iloc[i]/len(sens)
<ipython-input-105-e33da521c9b9>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['upper_case_p

,index_article,index_paragraph,content,label,unique_words_count,avg_unique_words_per_sentence,upper_case_pct,pos_tag_per_para,avg_pos_tag_per_sen,pos_to_unique_ratio,syllabus_pct,fk_grade
0,14,1,The prognostic assessment of breast cancer is ...,1,59,19.666667,3,16,5.333333,0.271186,1.945205,16.853425
1,14,2,"Currently, the most significant prognosticator...",1,80,16.000000,5,17,3.400000,0.212500,1.872549,14.462078
2,14,3,"Other major prognostic risk factors, especiall...",1,55,13.750000,3,15,3.750000,0.272727,1.950617,15.324784
3,14,4,It is known that invasive breast cancer (a mal...,1,98,24.500000,5,19,4.750000,0.193878,1.935714,20.901429
4,14,5,A total of 408 patients with primary invasive ...,1,126,15.750000,28,19,2.375000,0.150794,1.636364,13.371591
...,...,...,...,...,...,...,...,...,...,...,...,...
135,18,20,The miRNA expression profiles of BCSCs were ob...,0,92,18.400000,23,18,3.600000,0.195652,1.787402,15.407339
136,18,21,Bioinformatic analysis and prediction programs...,0,92,13.142857,19,16,2.285714,0.173913,1.832061,13.326892
137,18,22,The analysis of miR-21's target revealed two s...,0,72,14.400000,20,18,3.600000,0.250000,1.630435,10.825130
138,18,23,The identification of potential targets for di...,0,62,15.500000,20,17,4.250000,0.274194,1.752809,13.770646


conjunction


In [ ]:
def sentences_count(text):
    # 大小写字母集合的生成
    alpha = [chr(i) for i in range(ord('a'),ord('z')+1)]
    ALPHA = [chr(i) for i in range(ord('A'),ord('Z')+1)]
    A_S = alpha + ALPHA
    count = 0                       # 句子数记录
    sentence_list = text.split('.') # 使用'.'拆分句子  
    for s in sentence_list:
        for i in range(len(s)):
            if s[i] in A_S:
                count +=1
                break
    return count

def words_count(text):
  num_large_words = 0
  # string.punctuation
  punctuation_string = string.punctuation
  for i in punctuation_string:
    replaced_text = text.replace(i, '')
  word_list = replaced_text.split(" ")
  num_words = len(word_list)
  for word in word_list:
    if len(word) > 6:
      num_large_words += 1
  return num_words, num_large_words

def stopwords_count(text):
  num_stopwords = 0
  stop_words = set(stopwords.words('english')) 
  word_tokens = word_tokenize(text) 
  for w in word_tokens:
    if w in stop_words:
      num_stopwords += 1
  return num_stopwords 

def conjunction_count(text):
  tokenize_text = nltk.word_tokenize(text)
  tag_list = nltk.pos_tag(tokenize_text)
  cnj_count = 0
  for tag in tag_list:
    if tag[1] == "CC":
      cnj_count += 1
  return cnj_count

my_df['num_conjunctions'] = 0
my_df['avg_words_per_sentence'] = 0
my_df['avg_stopwords_per_sentence'] = 0
my_df['total_words'] = 0
my_df['num_large_words'] = 0
my_df['num_sentences'] = 0
my_df['avg_len_sentences'] = 0

for i in range(len(my_df)):
  text = my_df['content'].iloc[i]
  num_sentences = sentences_count(text)
  my_df['num_conjunctions'].iloc[i] = conjunction_count(text)
  my_df['avg_words_per_sentence'].iloc[i] = words_count(text)[0]/num_sentences
  my_df['avg_stopwords_per_sentence'].iloc[i] = stopwords_count(text)/num_sentences
  my_df['total_words'].iloc[i] = words_count(text)[0]
  my_df['num_large_words'].iloc[i] = words_count(text)[1]
  my_df['num_sentences'].iloc[i] = num_sentences
  my_df['avg_len_sentences'].iloc[i] = len(text)/num_sentences
my_df

<ipython-input-106-1cdbdb3d3614>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df['num_conjunctions'].iloc[i] = conjunction_count(text)
<ipython-input-106-1cdbdb3d3614>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df['avg_words_per_sentence'].iloc[i] = words_count(text)[0]/num_sentences
<ipython-input-106-1cdbdb3d3614>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df['avg_stopwords_per_sentence'].iloc[i] = stopwor

,index_article,index_paragraph,content,label,unique_words_count,avg_unique_words_per_sentence,upper_case_pct,pos_tag_per_para,avg_pos_tag_per_sen,pos_to_unique_ratio,syllabus_pct,fk_grade,num_conjunctions,avg_words_per_sentence,avg_stopwords_per_sentence,total_words,num_large_words,num_sentences,avg_len_sentences
0,1.0,1.0,Neurogenesis is a strictly controlled process ...,1.0,115,16.428571,15,18,2.571429,0.156522,1.994012,17.243627,8,23.857143,7.714286,167,71,7,170.857143
1,1.0,2.0,The potential of involving adult neurogenesis ...,1.0,120,20.000000,14,17,2.833333,0.141667,1.873563,17.828046,8,29.000000,9.833333,174,74,6,200.333333
2,1.0,3.0,To remedy this absence of suitable tool for ne...,1.0,62,20.666667,5,16,5.333333,0.258065,1.756098,15.791951,3,27.333333,10.000000,82,34,3,186.333333
3,1.0,4.0,A 2380-bp Sal I-Not I fragment of pCAG-CreERT2...,1.0,87,10.875000,51,18,2.250000,0.206897,1.643357,10.772858,4,15.888889,4.666667,143,49,9,114.777778
4,1.0,5.0,"The targeting plasmid, phuDCX-3509-CreERT2-3’-...",1.0,117,11.700000,49,14,1.400000,0.119658,1.589595,9.914225,4,15.727273,4.909091,173,57,11,104.545455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784,13.0,23.0,Microtiter plates from Nunc in Denmark were us...,0.0,100,20.000000,31,17,3.400000,0.170000,1.507576,12.495394,3,18.857143,6.857143,132,37,7,112.000000
785,13.0,24.0,Splenocytes from mice were collected after the...,0.0,41,13.666667,9,15,5.000000,0.365854,1.600000,9.790000,3,16.666667,6.666667,50,14,3,106.666667
786,13.0,25.0,"On day 3 after the final vaccination, the cerv...",0.0,57,14.250000,15,13,3.250000,0.228070,1.841270,12.279484,3,15.750000,4.250000,63,25,4,109.500000
787,13.0,26.0,After taking out the lungs of mice six days af...,0.0,93,15.500000,18,17,2.833333,0.182796,1.669565,11.585870,6,19.166667,6.666667,115,36,6,128.166667


In [ ]:
val_df['num_conjunctions'] = 0
val_df['avg_words_per_sentence'] = 0
val_df['avg_stopwords_per_sentence'] = 0
val_df['total_words'] = 0
val_df['num_large_words'] = 0
val_df['num_sentences'] = 0
val_df['avg_len_sentences'] = 0

for i in range(len(val_df)):
  text = val_df['content'].iloc[i]
  num_sentences = sentences_count(text)
  val_df['num_conjunctions'].iloc[i] = conjunction_count(text)
  val_df['avg_words_per_sentence'].iloc[i] = words_count(text)[0]/num_sentences
  val_df['avg_stopwords_per_sentence'].iloc[i] = stopwords_count(text)/num_sentences
  val_df['total_words'].iloc[i] = words_count(text)[0]
  val_df['num_large_words'].iloc[i] = words_count(text)[1]
  val_df['num_sentences'].iloc[i] = num_sentences
  val_df['avg_len_sentences'].iloc[i] = len(text)/num_sentences
val_df

<ipython-input-107-b83484587d40>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df['num_conjunctions'].iloc[i] = conjunction_count(text)
<ipython-input-107-b83484587d40>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df['avg_words_per_sentence'].iloc[i] = words_count(text)[0]/num_sentences
<ipython-input-107-b83484587d40>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df['avg_stopwords_per_sentence'].iloc[i] = stop

,index_article,index_paragraph,content,label,unique_words_count,avg_unique_words_per_sentence,upper_case_pct,pos_tag_per_para,avg_pos_tag_per_sen,pos_to_unique_ratio,syllabus_pct,fk_grade,num_conjunctions,avg_words_per_sentence,avg_stopwords_per_sentence,total_words,num_large_words,num_sentences,avg_len_sentences
0,15,1,"In the last two decades, corticomuscular synch...",1,39,19.500000,5,11,5.500000,0.282051,2.021739,17.236522,3,23.000000,7.000000,46,20,2,170.500000
1,15,2,Several investigations focused on the mechanis...,1,139,17.375000,24,20,2.500000,0.143885,1.800000,15.156250,6,24.375000,7.625000,195,74,8,166.750000
2,15,3,We addressed this question investigating the C...,1,36,36.000000,4,15,15.000000,0.416667,1.923077,22.312308,1,13.000000,3.666667,39,13,3,87.666667
3,15,4,"We tested the following predictions: First, ba...",1,48,48.000000,4,18,18.000000,0.375000,1.981481,28.851481,1,54.000000,20.000000,54,22,1,380.000000
4,15,5,"Second, recent studies on corticospinal intera...",1,55,18.333333,7,16,5.333333,0.290909,1.666667,13.046667,2,23.000000,9.333333,69,24,3,151.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,18,13,Tissue slides from the tumors were made and su...,0,42,14.000000,10,14,4.666667,0.333333,1.750000,12.860000,6,20.000000,6.333333,60,19,3,134.666667
138,18,14,The hybridisation reaction was conducted twice...,0,119,23.800000,31,16,3.200000,0.134454,1.626582,15.927671,5,22.571429,5.857143,158,42,7,140.285714
139,18,15,The study conducted real-time RT-PCR for 10 mi...,0,67,13.400000,25,15,3.000000,0.223881,1.582418,10.180527,5,18.200000,5.200000,91,39,5,125.600000
140,18,16,The miRNAs were analyzed through various onlin...,0,124,24.800000,52,18,3.600000,0.145161,1.886228,19.693485,10,33.400000,6.600000,167,79,5,256.400000


In [ ]:
test_df['num_conjunctions'] = 0
test_df['avg_words_per_sentence'] = 0
test_df['avg_stopwords_per_sentence'] = 0
test_df['total_words'] = 0
test_df['num_large_words'] = 0
test_df['num_sentences'] = 0
test_df['avg_len_sentences'] = 0

for i in range(len(test_df)):
  text = test_df['content'].iloc[i]
  num_sentences = sentences_count(text)
  test_df['num_conjunctions'].iloc[i] = conjunction_count(text)
  test_df['avg_words_per_sentence'].iloc[i] = words_count(text)[0]/num_sentences
  test_df['avg_stopwords_per_sentence'].iloc[i] = stopwords_count(text)/num_sentences
  test_df['total_words'].iloc[i] = words_count(text)[0]
  test_df['num_large_words'].iloc[i] = words_count(text)[1]
  test_df['num_sentences'].iloc[i] = num_sentences
  test_df['avg_len_sentences'].iloc[i] = len(text)/num_sentences
test_df

<ipython-input-108-035bca432aa3>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['num_conjunctions'].iloc[i] = conjunction_count(text)
<ipython-input-108-035bca432aa3>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['avg_words_per_sentence'].iloc[i] = words_count(text)[0]/num_sentences
<ipython-input-108-035bca432aa3>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['avg_stopwords_per_sentence'].iloc[i] = s

,index_article,index_paragraph,content,label,unique_words_count,avg_unique_words_per_sentence,upper_case_pct,pos_tag_per_para,avg_pos_tag_per_sen,pos_to_unique_ratio,syllabus_pct,fk_grade,num_conjunctions,avg_words_per_sentence,avg_stopwords_per_sentence,total_words,num_large_words,num_sentences,avg_len_sentences
0,14,1,The prognostic assessment of breast cancer is ...,1,59,19.666667,3,16,5.333333,0.271186,1.945205,16.853425,4,14.600000,4.800000,73,35,5,102.200000
1,14,2,"Currently, the most significant prognosticator...",1,80,16.000000,5,17,3.400000,0.212500,1.872549,14.462078,2,20.400000,6.200000,102,42,5,146.800000
2,14,3,"Other major prognostic risk factors, especiall...",1,55,13.750000,3,15,3.750000,0.272727,1.950617,15.324784,2,20.250000,5.500000,81,34,4,148.000000
3,14,4,It is known that invasive breast cancer (a mal...,1,98,24.500000,5,19,4.750000,0.193878,1.935714,20.901429,6,35.000000,13.000000,140,59,4,248.750000
4,14,5,A total of 408 patients with primary invasive ...,1,126,15.750000,28,19,2.375000,0.150794,1.636364,13.371591,7,18.000000,6.818182,198,68,11,113.727273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,18,20,The miRNA expression profiles of BCSCs were ob...,0,92,18.400000,23,18,3.600000,0.195652,1.787402,15.407339,4,25.400000,7.800000,127,43,5,170.600000
136,18,21,Bioinformatic analysis and prediction programs...,0,92,13.142857,19,16,2.285714,0.173913,1.832061,13.326892,3,18.714286,6.857143,131,57,7,128.142857
137,18,22,The analysis of miR-21's target revealed two s...,0,72,14.400000,20,18,3.600000,0.250000,1.630435,10.825130,6,18.400000,6.000000,92,27,5,120.800000
138,18,23,The identification of potential targets for di...,0,62,15.500000,20,17,4.250000,0.274194,1.752809,13.770646,6,22.250000,8.750000,89,30,4,144.750000


XGBOOST

In [ ]:
import xgboost as xgb

from sklearn.utils import shuffle
from sklearn import metrics



df = shuffle(my_df)
data_len = df['content'].str.len()
# train_data = df.iloc[:800,3:]
# test_data = df.iloc[800:,3:]

# X_train, X_val, y_train, y_val = train_test_split(train_data.iloc[:,1:],train_data['label'], test_size=0.2, random_state=2023)
X_train, y_train = df.iloc[:,4:],df['label']
X_val, y_val = val_df.iloc[:,4:],val_df['label']
X_test,y_test = test_df.iloc[:,4:],test_df['label']

model = xgb.XGBClassifier()
model.fit(X_train, y_train)

pred = model.predict(X_val)
# pred = model.predict(X_test)
print(metrics.classification_report(y_val, pred))
# print(metrics.confusion_matrix(y_val, pred))

              precision    recall  f1-score   support

           0       0.49      0.56      0.52        71
           1       0.48      0.41      0.44        71

    accuracy                           0.49       142
   macro avg       0.49      0.49      0.48       142
weighted avg       0.49      0.49      0.48       142



TF-IDF

In [ ]:
total = pd.concat([my_df,val_df,test_df])
total = total.reset_index(drop = True)
total

,index_article,index_paragraph,content,label,unique_words_count,avg_unique_words_per_sentence,upper_case_pct,pos_tag_per_para,avg_pos_tag_per_sen,pos_to_unique_ratio,syllabus_pct,fk_grade,num_conjunctions,avg_words_per_sentence,avg_stopwords_per_sentence,total_words,num_large_words,num_sentences,avg_len_sentences
0,1.0,1.0,Neurogenesis is a strictly controlled process ...,1.0,115,16.428571,15,18,2.571429,0.156522,1.994012,17.243627,8,23.857143,7.714286,167,71,7,170.857143
1,1.0,2.0,The potential of involving adult neurogenesis ...,1.0,120,20.000000,14,17,2.833333,0.141667,1.873563,17.828046,8,29.000000,9.833333,174,74,6,200.333333
2,1.0,3.0,To remedy this absence of suitable tool for ne...,1.0,62,20.666667,5,16,5.333333,0.258065,1.756098,15.791951,3,27.333333,10.000000,82,34,3,186.333333
3,1.0,4.0,A 2380-bp Sal I-Not I fragment of pCAG-CreERT2...,1.0,87,10.875000,51,18,2.250000,0.206897,1.643357,10.772858,4,15.888889,4.666667,143,49,9,114.777778
4,1.0,5.0,"The targeting plasmid, phuDCX-3509-CreERT2-3’-...",1.0,117,11.700000,49,14,1.400000,0.119658,1.589595,9.914225,4,15.727273,4.909091,173,57,11,104.545455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,18.0,20.0,The miRNA expression profiles of BCSCs were ob...,0.0,92,18.400000,23,18,3.600000,0.195652,1.787402,15.407339,4,25.400000,7.800000,127,43,5,170.600000
1024,18.0,21.0,Bioinformatic analysis and prediction programs...,0.0,92,13.142857,19,16,2.285714,0.173913,1.832061,13.326892,3,18.714286,6.857143,131,57,7,128.142857
1025,18.0,22.0,The analysis of miR-21's target revealed two s...,0.0,72,14.400000,20,18,3.600000,0.250000,1.630435,10.825130,6,18.400000,6.000000,92,27,5,120.800000
1026,18.0,23.0,The identification of potential targets for di...,0.0,62,15.500000,20,17,4.250000,0.274194,1.752809,13.770646,6,22.250000,8.750000,89,30,4,144.750000


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

X = total['content']
train_data = vectorizer.fit_transform(X)
word_list = vectorizer.get_feature_names_out() # 所有统计的词
print("统计词列表")
print(word_list)
weight_train = train_data.toarray().T
# weight_test = test_data.toarray()
print("train TFIDF权重")
print(weight_train)
iter = 0
for i in range(len(word_list)):
  if sum(weight_train[i]) < 20:
    continue
  else:
    total[word_list[i]] = 0
    iter += 1
    print(iter)
    for k in range(len(weight_train[i])):
      total[word_list[i]].iloc[k] = weight_train[i][k]


total

统计词列表
['00' '000' '00001' ... 'μg' 'μl' 'μm']
train TFIDF权重
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
1


<ipython-input-111-cfeeba85bc7b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total[word_list[i]].iloc[k] = weight_train[i][k]
<ipython-input-111-cfeeba85bc7b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total[word_list[i]].iloc[k] = weight_train[i][k]


2
3


<ipython-input-111-cfeeba85bc7b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total[word_list[i]].iloc[k] = weight_train[i][k]
<ipython-input-111-cfeeba85bc7b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total[word_list[i]].iloc[k] = weight_train[i][k]


4
5


...

12
13
14


<ipython-input-111-cfeeba85bc7b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total[word_list[i]].iloc[k] = weight_train[i][k]
<ipython-input-111-cfeeba85bc7b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total[word_list[i]].iloc[k] = weight_train[i][k]
<ipython-input-111-cfeeba85bc7b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total[word_list[i]].iloc[k] = weight_train[i][k]


15
16
17


<ipython-input-111-cfeeba85bc7b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total[word_list[i]].iloc[k] = weight_train[i][k]
<ipython-input-111-cfeeba85bc7b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total[word_list[i]].iloc[k] = weight_train[i][k]
<ipython-input-111-cfeeba85bc7b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total[word_list[i]].iloc[k] = weight_train[i][k]


,index_article,index_paragraph,content,label,unique_words_count,avg_unique_words_per_sentence,upper_case_pct,pos_tag_per_para,avg_pos_tag_per_sen,pos_to_unique_ratio,...,in,is,of,or,that,the,to,was,were,with
0,1.0,1.0,Neurogenesis is a strictly controlled process ...,1.0,115,16.428571,15,18,2.571429,0.156522,...,0.130602,0.059720,0.091563,0.000000,0.000000,0.131589,0.017546,0.000000,0.021552,0.000000
1,1.0,2.0,The potential of involving adult neurogenesis ...,1.0,120,20.000000,14,17,2.833333,0.141667,...,0.132499,0.030293,0.077411,0.058713,0.025443,0.163166,0.053402,0.000000,0.000000,0.000000
2,1.0,3.0,To remedy this absence of suitable tool for ne...,1.0,62,20.666667,5,16,5.333333,0.258065,...,0.052328,0.000000,0.171202,0.000000,0.040192,0.093729,0.056240,0.000000,0.000000,0.000000
3,1.0,4.0,A 2380-bp Sal I-Not I fragment of pCAG-CreERT2...,1.0,87,10.875000,51,18,2.250000,0.206897,...,0.027659,0.000000,0.077565,0.000000,0.000000,0.173400,0.029727,0.053843,0.054772,0.054188
4,1.0,5.0,"The targeting plasmid, phuDCX-3509-CreERT2-3’-...",1.0,117,11.700000,49,14,1.400000,0.119658,...,0.014612,0.053451,0.122928,0.025899,0.000000,0.157035,0.000000,0.094814,0.038580,0.133591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,18.0,20.0,The miRNA expression profiles of BCSCs were ob...,0.0,92,18.400000,23,18,3.600000,0.195652,...,0.113802,0.000000,0.106379,0.033619,0.029136,0.067947,0.081540,0.098460,0.025039,0.000000
1024,18.0,21.0,Bioinformatic analysis and prediction programs...,0.0,92,13.142857,19,16,2.285714,0.173913,...,0.087698,0.096243,0.081978,0.000000,0.080831,0.078542,0.000000,0.022763,0.000000,0.022909
1025,18.0,22.0,The analysis of miR-21's target revealed two s...,0.0,72,14.400000,20,18,3.600000,0.250000,...,0.042906,0.156957,0.080216,0.000000,0.032956,0.038427,0.023057,0.000000,0.000000,0.028020
1026,18.0,23.0,The identification of potential targets for di...,0.0,62,15.500000,20,17,4.250000,0.274194,...,0.112094,0.041005,0.062870,0.039737,0.068878,0.060235,0.024095,0.087284,0.029597,0.029281


In [ ]:
train_data = total.iloc[:746,:]
X_val, y_val = total.iloc[746:888,19:],total.loc[746:887,'label']
X_test,y_test = total.iloc[888:,19:],total.loc[888:,'label']

train_data = shuffle(train_data)

X_train = train_data.iloc[:,19:]
y_train = train_data['label']

model = xgb.XGBClassifier()
model.fit(X_train, y_train)
print(len(X_val))
print(len(y_val))

pred_val = model.predict(X_val)
pred_test = model.predict(X_test)
print('validation result')
print(metrics.classification_report(y_val, pred_val))
print('test result')
print(metrics.classification_report(y_test, pred_test))

# # print(metrics.confusion_matrix(y_val, pred))

142
142
validation result
              precision    recall  f1-score   support

         0.0       0.58      0.70      0.64        71
         1.0       0.62      0.49      0.55        71

    accuracy                           0.60       142
   macro avg       0.60      0.60      0.59       142
weighted avg       0.60      0.60      0.59       142

test result
              precision    recall  f1-score   support

         0.0       0.53      0.60      0.56        70
         1.0       0.54      0.47      0.50        70

    accuracy                           0.54       140
   macro avg       0.54      0.54      0.53       140
weighted avg       0.54      0.54      0.53       140

